In [26]:
import findspark
findspark.init()

In [27]:
from pyspark.sql import SparkSession

In [28]:
spark = SparkSession.builder.appName('pyspark-broadcast').getOrCreate()

In [62]:
countries = {'IND':"India",'AUS':"Australia", 'JPN':"JAPAN"}

In [63]:
broadcast_countries = spark.sparkContext.broadcast(countries)

In [64]:
data = [(100,'Debit',1000.0,'IND'),(101,"Credit",2000.0,'IND'),
        (102,'Debit',3000.0,'AUS'),(103,'Credit',4000.0,'JPN'),
        (104,'Debit',5000.0,'IND'),(105,'Credit',6000.0,'AUS')]

In [65]:
rdd = spark.sparkContext.parallelize(data)

In [66]:
rdd.collect()

[(100, 'Debit', 1000.0, 'IND'),
 (101, 'Credit', 2000.0, 'IND'),
 (102, 'Debit', 3000.0, 'AUS'),
 (103, 'Credit', 4000.0, 'JPN'),
 (104, 'Debit', 5000.0, 'IND'),
 (105, 'Credit', 6000.0, 'AUS')]

In [67]:
def country_convert(code):
    return broadcast_countries.value[code]

In [68]:
rdd1 = rdd.map(lambda x : (x[0], x[1], x[2], country_convert(x[3])))

In [69]:
rdd1.collect()

[(100, 'Debit', 1000.0, 'India'),
 (101, 'Credit', 2000.0, 'India'),
 (102, 'Debit', 3000.0, 'Australia'),
 (103, 'Credit', 4000.0, 'JAPAN'),
 (104, 'Debit', 5000.0, 'India'),
 (105, 'Credit', 6000.0, 'Australia')]

In [70]:
rdd2 = rdd1.groupBy(lambda x : (x[1], x[3]))

In [71]:
rdd2.collect()

[(('Credit', 'JAPAN'),
  <pyspark.resultiterable.ResultIterable at 0x2042c33b0a0>),
 (('Credit', 'Australia'),
  <pyspark.resultiterable.ResultIterable at 0x2042c33a950>),
 (('Debit', 'India'),
  <pyspark.resultiterable.ResultIterable at 0x2042c33a6e0>),
 (('Credit', 'India'),
  <pyspark.resultiterable.ResultIterable at 0x2042c33ae90>),
 (('Debit', 'Australia'),
  <pyspark.resultiterable.ResultIterable at 0x2042c33ad70>)]

In [72]:
print([(i, list(j)) for (i, j) in rdd2.take(1)])

[(('Credit', 'JAPAN'), [(103, 'Credit', 4000.0, 'JAPAN')])]


In [73]:
rdd3 = rdd2.map(lambda x : (x[0], len(x[1])))

In [74]:
rdd3.collect()

[(('Credit', 'JAPAN'), 1),
 (('Credit', 'Australia'), 1),
 (('Debit', 'India'), 2),
 (('Credit', 'India'), 1),
 (('Debit', 'Australia'), 1)]